Run the T2 sncosmo on sample ampel/ztf lightcurves

In [1]:
import re, os
#import json, os, sys, re, glob
import ampel.utils.json_serialization as ampel_serialization
import logging
import gzip
import importlib
from ampel.ztf.pipeline.common.ZTFUtils import ZTFUtils
#import sncosmo
#import matplotlib.pyplot as plt
#import numpy as np
import ampel.contrib.hu.t2.T2SNCosmo as T2SNCosmo
import sncosmo

In [2]:
importlib.reload(T2SNCosmo)

<module 'ampel.contrib.hu.t2.T2SNCosmo' from '/home/jnordin/github/ampeldev/Ampel-contrib-HU/ampel/contrib/hu/t2/T2SNCosmo.py'>

In [ ]:
# Find some sample data to use for fit
# Get some sample lcs
tvdir = '/home/jnordin/data/transientViews/'
fname = tvdir+'tnscomplete_aug28.json.gz'
# Get a TransientView iterator
if re.search('gz$',fname):
    tv_iterator = ampel_serialization.load(gzip.open(fname,'rb'))
else:    
    tv_iterator = ampel_serialization.load(open(fname))
tvlist = []
try:
    for tv in tv_iterator:
        tvlist.append(tv)
except EOFError as e:
    print('... prob just the usual missing eof marker. Check that the number agrees')
print('Found %s transient views'%(len(tvlist)))

In [4]:
# Create a logger
logger = logging.getLogger('jnordin')
logger.setLevel((logging.INFO))
logpath = '/home/jnordin/tmp/'
handler = logging.FileHandler(os.path.join(logpath, 'run_t2sncosmo.log'))
logger.addHandler(handler)

In [ ]:
# Base definition for salt2 fit
base_config = {}
run_config = {
            "model": "salt2",
            "with_upper_limits": False,
            "sncosmo_kwarg": {},
            "min_det" : 8,
            "jd_reject_sigma" : 5,
            "min_duration" : 10,
            "max_duration" : 90
        }
fit_sne = []

In [ ]:
# Base definition for kilonova fit
base_config = {}
run_config = {
            "model": "kilo_mej0.02_lant15_theta90",
            "with_upper_limits": False,
#            "sncosmo_kwarg": {"bounds":{'z':[0,0.05]}},
#            "sncosmo_kwarg": {"bounds":{'amplitude':[2e-14,1e-13],'z':[0,0.05]}},
#            "sncosmo_kwarg": {"bounds":{'amplitude':[4e-13,8e-13],'t0':[2458713.75,2458713.751],'z':[0,0.05]}},
#            "sncosmo_kwarg": {"bounds":{'t0':[2458713.75,2458713.751],'z':[0,0.05]}},
#            "sncosmo_kwarg": {"bounds":{'amplitude':[1500,3000]}},
            "sncosmo_kwarg": {},
#            "z_discrete": [0.01],
            "min_det" : 8,
            "jd_reject_sigma" : 5,
            "min_duration" : 3,
            "max_duration" : 16
        }

In [3]:
importlib.reload(sncosmo)
importlib.reload(T2SNCosmo)
myt2 = T2SNCosmo.T2SNCosmo(logger, base_config)

NameError: name 'logger' is not defined

In [ ]:
%%time
for tv in tvlist:
    ztf_name = ZTFUtils.to_ztf_id(tv.tran_id)    
    if len(fit_sne)>0 and not ztf_name in fit_sne : continue
    print(ztf_name)    
    run_config['savefile'] = '/home/jnordin/tmp/sncosmo/%s.pdf'%(ztf_name)
#    run_config['savefile'] = None
    lc = tv.get_latest_lightcurve()
    out = myt2.run(lc, run_config)
    try:
        print(out['fit_results'])
    except KeyError:
        print('no result')

In [ ]:
# Focus on subset of sne where fit aught to work
# These are sne that seem to fit both for salt and kilonova fits
fit_sne = ["ZTF19abqhbuk","ZTF19abqykti"]
# These are sne that work for salt - do they work for the kilonova?
fit_sne = ["ZTF19abjgdko","ZTF19ablpfhz","ZTF19aboumcn","ZTF19abqgxpt","ZTF19abqyopw","ZTF19abrorbx"]
fit_sne = ["ZTF19abqgxpt"]
# Objects that just seems to lead to too low fits
fit_sne = ["ZTF19abqyogn","ZTF19abqyoom"]
# Next round, after increasing the tgrid sampling. Still has not fixed all..
fit_sne = ["ZTF19abqgyxp","ZTF19abqykti","ZTF19abttlwl"]
fit_sne = ["ZTF19abqgyxp"]

#fit_sne = []

In [ ]:
# Manual run
lc = None
for tv in tvlist:
    ztf_name = ZTFUtils.to_ztf_id(tv.tran_id)    
    if not ztf_name in fit_sne : continue
    print(ztf_name)    
    lc = tv.get_latest_lightcurve()
print(lc)

In [ ]:
# Load the data
sncosmo_tool = T2SNCosmo.SNCosmoTool()
sncosmo_tool.load_data(lc, with_upper_limits = False )

In [ ]:
# Load the model
sncosmo_model, parameter_names, fit_kwargs = sncosmo_tool.get_kilonova_model(
        "kilo_mej0.02_lant15_theta90", 0)

In [ ]:
sncosmo_model.set(z=0)
sncosmo_model.set(hostebv=0)
sncosmo_model.set(theta=90)

In [ ]:
result, fit_model = sncosmo.fit_lc(sncosmo_tool.sncosmo_data, sncosmo_model, ['amplitude','t0'])

In [ ]:
result

In [ ]:
print(result['success'])
print(result['chisq'])
fig = sncosmo.plot_lc(sncosmo_tool.sncosmo_data, model = fit_model, pulls=True)

In [ ]:
for amp in [200,220,240,260,280,"300"]:
    sncosmo_model.set(amplitude=amp)
    result, fit_model = sncosmo.fit_lc(sncosmo_tool.sncosmo_data, sncosmo_model, ['t0'])
    print(amp)
    print(result['success'])
    print(result['chisq'])
    fig = sncosmo.plot_lc(sncosmo_tool.sncosmo_data, model = fit_model, pulls=True)